In [19]:
## Full backend analysis for the CREEDS and L1000 dataset 
import os
import pandas as pd
import itertools
from pandas.compat import StringIO
import numpy as np
from numpy import loadtxt
import sys
import json
from pprint import pprint
import objectpath
import csv
import re
import matplotlib.pyplot as plt
import json, requests
from pprint import pprint
import scipy
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances
from clustergrammer_widget import *
def get_geneset(df, indexer):
    df_ = df.loc[indexer, :]
    return list(df_[df_ == 1].index)


## load in the pre-formed datasets from the L1000_Analysis and CREEDS_Analysis files.
## THIS TAKES A WHILE TO LOAD, SO ONLY LOAD THIS ONCE AND EARLY

# L1000 up and down gene loads for drug signatures
L1000_up_genes = pd.read_csv("L1000_up_genes.csv")
L1000_down_genes = pd.read_csv("L1000_down_genes.csv")

# CREEDS up and down genes for disease signatures
with open("disease_signatures-v1.0.json") as f:
    CREEDS_data = json.load(f)

# generate the up and down gene signatures
#CREEDS_up_genes = {
#    row['do_id']: row['up_genes']
#    for row in CREEDS_data
#}
#CREEDS_down_genes = {
#    row['do_id']: row['down_genes']
#    for row in CREEDS_data
#}

# RETURNS THE do_id, geo_id, and disease name in a dictionary
CREEDS_GSE = {
    row['id']: [row['geo_id'], row["disease_name"]]
    for row in CREEDS_data
}

# load in the EMR Data (filtered > 200 in R code [Drug_diagnosis_test_code.R])
EMR_data = pd.read_csv("EMR_greater_200.csv")
## subset EMR data by the DOI and/or DrOI
EMR_data_df = pd.DataFrame(EMR_data)
#EMR_data
EMR_data_df.drop(EMR_data_df.columns[[0]], axis = 1, inplace = True) # remove the unecessary columns
#EMR_data_df

# implement the search from ICD9-do_id from the manual conversion
icd9_to_doid = pd.read_csv("ICD9_CREEDS_conversion.csv")
icd9_to_doid = pd.DataFrame(icd9_to_doid) # convert it to a data fram to drop unecessary rows
#icd9_to_doid # sanity check
icd9_to_doid_final = icd9_to_doid.drop(icd9_to_doid.columns[[0, 6, 7, 8, 9, 10, 11, 12, 13, 14]], axis = 1)
#icd9_to_doid_final # sanity check


posssible_disease_list = set(icd9_to_doid_final.Disease) 
#& set(EMR_data_df.Description) # will return {"Alzheimer's disease", "Barrett's esophagus", 'Dehydration', 'Sepsis'}

posssible_disease_list = list(posssible_disease_list) # this will be what the dropdown should display
###### GENERATE THE FULL ANALYSIS BETWEEN CREEDS AND L1000 GIVEN A DISEASE OF INTEREST

# USER INPUT
####
DOI = "asthma" # disease of interest. CAN TAKE FROM posssible_disease_list FOR NOW
####
#possible_diseases = EMR_data_df["Description"] #possible diseases from the Sinai EMR

### DISEASE --> ICD9 --> DOI
# get the ICD9 from the DOI
DOI_ICD9 = icd9_to_doid_final[icd9_to_doid_final.Disease.apply(lambda s: bool(re.compile(DOI, re.IGNORECASE).search(s)))]
DOI_ICD9_codes = DOI_ICD9.ICD9

# get the do_id from the DOI
DOI_DOID_codes = DOI_ICD9.DOID

# get the do_id from the DOI
DOI_CREEDS_codes = DOI_ICD9.CREEDS_drug_id

# INPUT SIMILAR CREEDS CODE FROFM THE DRUG_QUERY

#DOIDs_up = {
#    doid: geneset
#    for doid, geneset in CREEDS_up_genes.items()
#    if doid in set(DOI_DOID_codes)
#}

#DOIDs_down = {
#    doid: geneset
#    for doid, geneset in CREEDS_down_genes.items()
#    if doid in set(DOI_DOID_codes)
#}
    
##filter by DOI. Need to convert DOI to ICD9 first.
icd9_to_doid_final_search = icd9_to_doid_final[icd9_to_doid_final["Disease"].apply(lambda s: bool(re.compile(DOI, re.IGNORECASE).search(s)))]
icd9_to_doid_final_search1 = icd9_to_doid_final_search["ICD9"]

## rebuild the wildcard dataframe
icd9_wildcard = pd.DataFrame(icd9_to_doid_final_search1, columns=['ICD9'])
icd9_wildcard['ICD9_wildcard'] = icd9_wildcard['ICD9'].apply(lambda code: str(code).split('.')[0])
icd9_wildcard.head()

icd9_to_doid_final['ICD9_wildcard'] = icd9_to_doid_final['ICD9'].apply(lambda code: str(code).split('.')[0])
#icd9_to_doid_final.head()
ICD9_df_joined = pd.merge(
    left=icd9_wildcard, left_on='ICD9_wildcard',
    right=icd9_to_doid_final, right_on='ICD9_wildcard',
    how='inner',
    suffixes=(
        '_Manual',
        '_right',
    )
)


#ICD9_codes = str(int(ICD9_df_joined["ICD9_wildcard"].unique())) 
## generate an emr based on the ICD_9 codes extracted; can now extract the drug names as well
#emr_sub = EMR_data_df[EMR_data_df['ICD9'].apply(lambda s: bool(re.compile(ICD9_codes, re.IGNORECASE).search(s)))]
emr_sub = EMR_data_df[EMR_data_df["Description"].apply(lambda s: bool(re.compile(str(DOI), re.IGNORECASE).search(str(s))))]
#emr_sub[0:10]
emr_sub.reset_index(drop = True, inplace = True)

emr_sub = []
for a in ICD9_df_joined.ICD9_wildcard.unique():
    emr_sub1 = EMR_data_df[EMR_data_df["ICD9"].apply(lambda s: bool(re.compile(str(a), re.IGNORECASE).search(str(s))))]
    emr_sub.append(emr_sub1)
emr_sub_df = pd.concat(emr_sub)
#### L1000 integration
# disease to drug conversion (disease input)
top_drugs_from_disease = list(emr_sub_df.Drug_Name[0:10]) #take the top 5 drugs
test = pd.DataFrame(L1000_down_genes)
#for x in top_drugs_from_disease:
#    print(x)
#    top_drug_disease_extract = test[test['Unnamed: 0'].apply(lambda s: bool(re.compile(str(x), re.IGNORECASE).search(str(s))))]
metadata = pd.read_csv("L1000_metadata.csv")
metadata ## same as LINC1000h5.row_metadata_df
#metadata
for a in top_drugs_from_disease:
    print(a)
    meta_doi = metadata[metadata["pert_desc"].apply(lambda s: bool(re.compile(str(a), re.IGNORECASE).search(str(s))))]
meta_doi
meta_doi_ids = meta_doi.rid
query = list(meta_doi_ids)
#print(query)
# disease to drug conversion (disease input)
top_drugs_from_disease = list(emr_sub_df.Drug_Name[0:20]) #take the top 5 drugs
test = pd.DataFrame(L1000_down_genes)
top_drugs_from_disease


single_word_drugs = []
for i in top_drugs_from_disease:
    j = str(i)
    splitted = j.split()
    first_word = splitted[0]
   # print(first_word)
    single_word_drugs.append(first_word) 
#print(single_word_drugs)
single_word_drugs = list(set(single_word_drugs))
#single_word_drugs

# Generate a blacklist process
def process_blacklist(s):
    import re
    blacklist = [
        # remove the classifications of the drugs
        re.compile(r'INJ', re.IGNORECASE),
        re.compile(r'CAP', re.IGNORECASE),
        re.compile(r'\d+', re.IGNORECASE),
        
        # remove drugs that aren't in the L1000
        re.compile(r'SODIUM', re.IGNORECASE),
        re.compile(r'HEPATITIS', re.IGNORECASE),
        re.compile(r'HEPARIN', re.IGNORECASE),
        re.compile(r'CALCIUM', re.IGNORECASE),
        
    ]
    for b in blacklist:
        s = re.sub(b, '', s)
    return s.strip()
single_word_drugs_list = list(pd.Series(single_word_drugs).map(process_blacklist))
single_word_drugs_list


## L1000 API Integration
L1000FWD_URL = 'http://amp.pharm.mssm.edu/L1000FWD/'


L1000_reverse_drugs_store = []


for a in single_word_drugs_list:
    query_string = a
    L1000_reverse_drug_response = requests.get(L1000FWD_URL + 'synonyms/' + query_string)
    if L1000_reverse_drug_response.status_code == 200:
        #pprint(L1000_reverse_drug_response.json())
        L1000_reverse_significant_query = L1000_reverse_drug_response.json()
        if len(L1000_reverse_significant_query) > 0:
            json.dump(L1000_reverse_drug_response.json(), open(query_string + '_L1000_reverse_drug_query.json', 'w'), indent=4)
            L1000_reverse_significant_query = L1000_reverse_drug_response.json()
            L1000_reverse_significant_query_df = pd.DataFrame(L1000_reverse_significant_query)
            L1000_reverse_drugs_store.append(a)
            print("Found significant L1000 drug signatures for " + query_string)
        else:
            print("No significant L1000 drug signatures for " + query_string)
  
    
    
############################################################################################################################################
##################################################################################################
## X2K API integration 

# Import modules
import http.client
import json

##### Function to run X2K
### Input: a Python list of gene symbols
### Output: a dictionary containing the results of X2K, ChEA, G2N, KEA.


def run_X2K(input_genes, options={}):
    # Open HTTP connection
    conn = http.client.HTTPConnection("amp.pharm.mssm.edu")

    # Set default options
    default_options = {'text-genes': '\n'.join(input_genes),
                       'included_organisms': 'both',
                       'TF-target gene background database used for enrichment': 'ChEA & ENCODE Consensus',
                       'sort transcription factors by': 'p-value',
                       'min_network_size': 10,
                       'number of top TFs': 10,
                       'path_length': 2,
                       'min_number_of_articles_supporting_interaction': 0,
                       'max_number_of_interactions_per_protein': 200,
                       'max_number_of_interactions_per_article': 100,
                       'enable_BioGRID': True,
                       'enable_IntAct': True,
                       'enable_MINT': True,
                       'enable_ppid': True,
                       'enable_Stelzl': True,
                       'kinase interactions to include': 'kea 2018',
                       'sort kinases by': 'p-value'}

    # Update options
    for key, value in options.items():
        if key in default_options.keys() and key != 'text-genes':
            default_options.update({key: value})

    # Get payload
    boundary = "----WebKitFormBoundary7MA4YWxkTrZu0gW"
    payload = ''.join(
        ['--' + boundary + '\r\nContent-Disposition: form-data; name=\"{key}\"\r\n\r\n{value}\r\n'.format(**locals())
         for key, value in default_options.items()]) + '--' + boundary + '--'

    # Get Headers
    headers = {
        'content-type': "multipart/form-data; boundary=" + boundary,
        'cache-control': "no-cache",
    }

    # Initialize connection
    conn.request("POST", "/X2K/api", payload, headers)

    # Get response
    res = conn.getresponse()

    # Read response
    data = res.read().decode('utf-8')

    # Convert to dictionary
    x2k_results = {key: json.loads(value) if key != 'input' else value for key, value in json.loads(data).items()}

    # Clean results
    x2k_results['ChEA'] = x2k_results['ChEA']['tfs']
    x2k_results['G2N'] = x2k_results['G2N']['network']
    x2k_results['KEA'] = x2k_results['KEA']['kinases']
    x2k_results['X2K'] = x2k_results['X2K']['network']

    # Return results
    return x2k_results
####################################################################################################################################
### CREEDS DRUG CARD 
while("" in single_word_drugs_list):
    single_word_drugs_list.remove("")
    
single_word_drugs_list = "PREDNISONE"
CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'

CREEDS_drug_from_disease_up_genes = []
CREEDS_drug_from_disease_down_genes = []

for a in single_word_drugs_list:
    CREEEDS_Drug_response = requests.get(CREEDS_URL + 'search', params={'q':str(a)})
    if CREEEDS_Drug_response.status_code == 200:
        #pprint(CREEEDS_Drug_response.json())
        #json.dump(CREEEDS_Drug_response.json(), open(DrOI + '_api1_result.json', 'w'), indent=4)
        CREEDS_drug_output_df = pd.DataFrame(CREEEDS_Drug_response.json())
        
        if len(CREEDS_drug_output_df) > 0:
            CREEDS_drug_output_ids = list(CREEDS_drug_output_df["id"])
            print("CREEDS IDs found for " + a)
            for a in CREEDS_drug_output_ids:
                CREEDS_drug_sigs_response = requests.get(CREEDS_URL + 'api', params={'id':'drug:DM609'})
                if CREEDS_drug_sigs_response.status_code == 200:
                    CREEDS_drug_sigs_response_json = CREEDS_drug_sigs_response.json()

                    ## up genes
                    CREEDS_drug_sigs_up_genes = CREEDS_drug_sigs_response_json['up_genes']
                    CREEDS_drug_sigs_up_genes_df = pd.DataFrame(CREEDS_drug_sigs_up_genes) # this is the up genes dataframe
                    CREEDS_drug_sigs_up_genes_df.columns= ["Up_Genes", "Score"]
                    CREEDS_drug_sig_up_genes_list = list(CREEDS_drug_sigs_up_genes_df.Up_Genes)
                    filename1 = (a + "_CREEDS_drug_sig_up_genes.csv")
                    #CREEDS_drug_sigs_up_genes_df.to_csv(filename1) # this saves the df as a csv

                    ## down genes
                    CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes']
                    CREEDS_drug_sigs_down_genes_df = pd.DataFrame(CREEDS_drug_sigs_down_genes)# this is the down genes dataframe
                    filename2 = (a + "_CREEDS_drug_sig_down_genes.csv")
                    CREEDS_drug_sigs_down_genes_df.columns= ["Down_Genes", "Score"]
                    CREEDS_drug_sig_down_genes_list = list(CREEDS_drug_sigs_down_genes_df.Down_Genes)
                    #print(filename2)
                    #CREEDS_drug_sigs_down_genes_df.to_csv(filename2)
                    #CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes'] # this saves the df as a csv

                    ## json propagation
                    #pprint(response.json())
                    #json.dump(response.json(), open(a + '_CREEDS_Drug_sig.json', 'w'), indent=4) # if the user wants the entire json, they can download this

                    CREEDS_drug_from_disease_up_genes.append(CREEDS_drug_sigs_up_genes)
                    CREEDS_drug_from_disease_down_genes.append(CREEDS_drug_sigs_down_genes_df)
                    
                    
                     
                    ## X2K implementation

                    ## up genes
                    CREEDS_X2K_up_genes = run_X2K(CREEDS_drug_sig_up_genes_list)
                    CREEDS_X2K_up_genes = CREEDS_X2K_up_genes["X2K"]
                    CREEDS_X2K_up_genes_df = pd.DataFrame(CREEDS_X2K_up_genes['nodes'])
                    filename_up = (a + "_CREEDS_X2K_up_genes.csv")
                    #CREEDS_X2K_up_genes_df.to_csv(filename_up) # THIS IS THE FILE THEY SHOULD BE ABLE TO DOWNLOAD

                    ## down genes
                    CREEDS_X2K_down_genes = run_X2K(CREEDS_drug_sig_down_genes_list)
                    CREEDS_X2K_down_genes = CREEDS_X2K_down_genes["X2K"]
                    CREEDS_X2K_down_genes_df = pd.DataFrame(CREEDS_X2K_down_genes['nodes'])
                    filename_down = (a + "_CREEDS_X2K_down_genes.csv")
                    CREEDS_X2K_down_genes_df.to_csv(filename_down) # THIS IS THE FILE THEY SHOULD BE ABLE TO DOWNLOAD
                    
                  
        else:
            print ("No CREEDS IDs found for " + a)


ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER
ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) NEB SOLUTION
HEPARIN SODIUM INJ
DOCUSATE SODIUM 100 MG CAPSULE
ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) NEB SOLUTION
DOCUSATE SODIUM CAP
HEPARIN (PORCINE) 5,000 UNIT/ML INJECTION
SODIUM CHLORIDE 0.9%
ZZ IMS TEMPLATE
ALBUTEROL 90 MCG/ACTUATION AEROSOL INHALER
No significant L1000 drug signatures for ZZ
Found significant L1000 drug signatures for ALBUTEROL
Found significant L1000 drug signatures for ASPIRIN
No significant L1000 drug signatures for OXYCODONE-ACETAMINOPHEN
No significant L1000 drug signatures for Fentanyl
Found significant L1000 drug signatures for ESOMEPRAZOLE
Found significant L1000 drug signatures for PREDNISONE
Found significant L1000 drug signatures for IPRATROPIUM
Found significant L1000 drug signatures for Propofol
Found significant L1000 drug signatures for MONTELUKAST
Found significant L1000 drug signatures for DOCUSATE


In [17]:
CREEDS_drug_sigs_up_genes_df.columns= ["Up_Genes", "Score"]
CREEDS_drug_sigs_up_genes_df.Up_Genes

0                      Fgf2
1                     Smad5
2                      Ibsp
3                     Penk1
4                     Sert1
5                      Fth1
6          Pnpla2_predicted
7                     Hmga2
8                       Fos
9                     Ccng1
10                    Acadl
11                     Cd36
12                    Cpt1b
13                     Araf
14                      Ckm
15     RGD1564596_predicted
16                     Pklr
17                    Sepp1
18                   Atp5g2
19                     Bag3
20                    Cox5a
21                    Nisch
22                     Rpl6
23                   Ndufa5
24                      Dsp
25     RGD1305801_predicted
26         Sorbs1_predicted
27     RGD1565641_predicted
28                   Atp5c1
29                    Prph1
               ...         
278        Ankmy2_predicted
279    RGD1561878_predicted
280                    Stx7
281                    Bzw2
282                 

In [ ]:
\